In [1]:
import sys
import os

BASE_PATH = os.getenv('PROJECTPATH')

# Add the src directory to the system path
src_path = os.path.join(BASE_PATH, 'src')
sys.path.append(src_path)

In [2]:
import great_expectations as gx
import pandas as pd
import numpy as np
import os
from data import preprocess_data, read_datastore

In [3]:
df, version = read_datastore()
for col in df:
    print(df[col])

access_token is expired. Now: 2024-07-24 07:26:35.935981, token_expiry: 2024-07-24 07:19:21
Refreshing access_token
/home/microwaved_frog/MLOps-for-music-popularity-prediction/.venv/lib/python3.10/site-packages/httplib2/__init__.py:147: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  context = ssl.SSLContext(DEFAULT_TLS_VERSION)

0            1288.0
1            7871.0
2           14059.0
3            2471.0
4         1472617.0
            ...    
179935      37474.0
179936        478.0
179937      40725.0
179938      10523.0
179939      43895.0
Name: artist_followers, Length: 179940, dtype: float64
0                                                      []
1                             ['progressive alternative']
2                                         ['pet calming']
3                                         ['piano cover']
4                           ['filmi', 'modern bollywood']
                               ...                       
179935                      ['deep house

In [4]:
X, y = preprocess_data(df)
X.head()

/home/microwaved_frog/MLOps-for-music-popularity-prediction/src/data.py:374: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[col] = features[col].astype(expected_type)

/home/microwaved_frog/MLOps-for-music-popularity-prediction/src/data.py:352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[col] = features[col].apply(lambda x: re.sub(r'\W+', ' ', x)).apply(str.split)



,energy,valence,acousticness,instrumentalness,chart,mode,artist_followers,album_total_tracks,artist_popularity,tempo,...,available_markets_UZ,available_markets_VC,available_markets_VE,available_markets_VN,available_markets_VU,available_markets_WS,available_markets_XK,available_markets_ZA,available_markets_ZM,available_markets_ZW
0,0.345,0.1560,0.770080,0.825,0.0,0.0,-0.231849,-0.554068,-0.070729,1.338886,...,1,1,1,1,1,1,1,1,1,1
1,0.845,0.2950,0.000017,0.727,0.0,0.0,-0.230750,-0.146350,-1.101348,0.028699,...,1,1,1,1,1,1,1,1,1,1
2,0.102,0.0382,0.958835,0.786,0.0,0.0,-0.229717,0.628316,0.959889,-1.509792,...,1,1,1,1,1,1,1,1,1,1
3,0.014,0.0886,0.996988,0.901,0.0,1.0,-0.231652,-0.187121,-0.463346,-1.620670,...,1,1,1,1,1,1,1,1,1,1
4,0.501,0.6600,0.667671,0.000,0.0,0.0,0.013836,-0.309437,0.469118,1.034919,...,1,1,1,1,1,1,1,1,1,1


In [5]:
import great_expectations as gx
from great_expectations.data_context import FileDataContext
context = gx.get_context(context_root_dir=f"{BASE_PATH}/services/gx")
context

FileDataContext loading fluent config
Loading 'datasources' ->
[{'assets': [...], 'name': 'sample', 'type': 'pandas'},
 {'assets': [...], 'name': 'sample_data', 'type': 'pandas'},
 {'assets': [...], 'name': 'features', 'type': 'pandas'}]
_PandasDataAsset.dict() - missing config_provider, skipping config substitution
_PandasDataAsset.dict() - missing config_provider, skipping config substitution
_PandasDataAsset.dict() - missing config_provider, skipping config substitution
Loaded 'sample' from fluent config
Loaded 'sample_data' from fluent config
Loaded 'features' from fluent config


{
  "anonymous_usage_statistics": {
    "explicit_id": true,
    "usage_statistics_url": "https://stats.greatexpectations.io/great_expectations/v1/usage_statistics",
    "enabled": true,
    "explicit_url": false,
    "data_context_id": "658ab671-7357-4b71-8760-718815eecf13"
  },
  "checkpoint_store_name": "checkpoint_store",
  "config_variables_file_path": "uncommitted/config_variables.yml",
  "config_version": 3.0,
  "data_docs_sites": {
    "local_site": {
      "class_name": "SiteBuilder",
      "show_how_to_buttons": true,
      "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": "uncommitted/data_docs/local_site/"
      },
      "site_index_builder": {
        "class_name": "DefaultSiteIndexBuilder"
      }
    }
  },
  "datasources": {},
  "evaluation_parameter_store_name": "evaluation_parameter_store",
  "expectations_store_name": "expectations_store",
  "fluent_datasources": {
    "sample": {
      "type": "pandas",
      "assets":

In [6]:
data_asset = context.get_datasource("features").get_asset("features_dataframe")
batch_request = data_asset.build_batch_request(dataframe=X)

context.add_or_update_expectation_suite("features_expectations")

context.list_expectation_suites()

[ExpectationSuiteIdentifier::expectation_suite,
 ExpectationSuiteIdentifier::features_expectations]

In [7]:
for i in X.columns:
    print(i, X[i].nunique())

energy 2754
valence 1820
acousticness 5046
instrumentalness 5401
chart 2
mode 2
artist_followers 37407
album_total_tracks 243
artist_popularity 93
tempo 76116
speechiness 1569
danceability 1313
liveness 1744
loudness 29910
key_0.0 2
key_1.0 2
key_2.0 2
key_3.0 2
key_4.0 2
key_5.0 2
key_6.0 2
key_7.0 2
key_8.0 2
key_9.0 2
key_10.0 2
key_11.0 2
time_signature_0.0 2
time_signature_1.0 2
time_signature_3.0 2
time_signature_4.0 2
time_signature_5.0 2
album_release_date_year 113
album_release_date_month_sin 11
album_release_date_month_cos 11
album_release_date_day_sin 31
album_release_date_day_cos 26
album_release_date_weekday_sin 7
album_release_date_weekday_cos 7
n_vec_0 171178
n_vec_1 171183
n_vec_2 171182
n_vec_3 171181
n_vec_4 171183
n_vec_5 171184
n_vec_6 171177
n_vec_7 171181
n_vec_8 171176
n_vec_9 171183
g_vec_0 19468
g_vec_1 19468
g_vec_2 19468
g_vec_3 19468
g_vec_4 19468
g_vec_5 19468
g_vec_6 19468
g_vec_7 19468
g_vec_8 19468
g_vec_9 19468
explicit 1
available_markets_AD 2
availabl

In [8]:
from utils import init_hydra
cfg = init_hydra()
cfg

{'data': {'dataset_name': 'music_popularity', 'num_samples': 5, 'path_to_raw': './data/raw/tracks.csv', 'remote': 'mlops_remote', 'repo': '.', 'sample_num': 5, 'version': 'AIRFLOW2.4', 'target_features': ['popularity'], 'genres_feature': 'genres', 'multilabel_features': ['available_markets'], 'categorical_features': ['key', 'time_signature'], 'normal_features': ['artist_followers', 'album_total_tracks', 'artist_popularity', 'tempo', 'speechiness', 'danceability', 'liveness', 'loudness'], 'uniform_features': ['energy', 'valence', 'acousticness', 'instrumentalness', 'chart', 'mode'], 'convert_to_bool': ['explicit'], 'timedate_features': ['album_release_date'], 'low_features_number': ['track_id', 'streams', 'track_artists', 'added_at', 'track_album_album', 'duration_ms', 'track_track_number', 'rank', 'region', 'trend'], 'text_features': ['album_name', 'name'], 'missing_list': ['genres', 'available_markets'], 'missing_strings': ['album_name', 'name']}, 'docker_port': 5152, 'mae_threshold':

In [13]:
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name="features_expectations"
)

for col in X.columns:
    # All features should be non-null
    validator.expect_column_values_to_not_be_null(column=col)

    # If feature in uniform, must be non-negative float
    if col in cfg.data.uniform_features:
        validator.expect_column_values_to_be_of_type(column=col, type_="float")
        validator.expect_column_min_to_be_between(column=col, min_value=0, max_value=None)

    # If feature in normal, must be float
    elif col in cfg.data.normal_features:
        validator.expect_column_values_to_be_of_type(column=col, type_="float")

    # If feature is <something>_vec_<something>, must be float
    elif "vec" in col:
        validator.expect_column_values_to_be_of_type(column=col, type_="float")

    # If feature ends with _sin or _cos, must be float between -1 and 1 inclusive
    elif col.endswith("_sin") or col.endswith("_cos"):
        validator.expect_column_values_to_be_of_type(column=col, type_="float")
        validator.expect_column_min_to_be_between(column=col, min_value=-1, max_value=1)

    # If feature name starts with name from cfg.data.multilabel_features, must be 0 or 1
    elif any([col.startswith(i) for i in cfg.data.multilabel_features]):
        validator.expect_column_values_to_be_in_set(column=col, value_set=[0, 1])

    # If feature name starts with name from cfg.data.categorical_features, must be 0 or 1
    elif any([col.startswith(i) for i in cfg.data.categorical_features]):
        validator.expect_column_values_to_be_in_set(column=col, value_set=[0, 1])

    # If feature ends with year, must be integer
    elif col.endswith("year"):
        validator.expect_column_values_to_be_of_type(column=col, type_="int")

    

    


/home/microwaved_frog/MLOps-for-music-popularity-prediction/.venv/lib/python3.10/site-packages/great_expectations/expectations/expectation.py:1518: UserWarning: result_format configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(



Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

In [14]:
validator.save_expectation_suite(discard_failed_expectations=False)

	499 expectation(s) included in expectation_suite.


In [15]:
checkpoint = context.add_or_update_checkpoint(
    name = "features_test_checkpoint",
    validations = [
        {
            "batch_request": batch.batch_request,
            "expectation_suite_name": "features_expectations"
        } for batch in data_asset.get_batch_list_from_batch_request(batch_request)
    ]
)

In [16]:
# Print passed expectations
res = checkpoint.run()
res

	499 expectation(s) included in expectation_suite.


Calculating Metrics:   0%|          | 0/2245 [00:00<?, ?it/s]

{
  "run_id": {
    "run_name": null,
    "run_time": "2024-07-24T10:31:14.373785+03:00"
  },
  "run_results": {
    "ValidationResultIdentifier::features_expectations/__none__/20240724T073114.373785Z/features-features_dataframe": {
      "validation_result": {
        "success": true,
        "results": [
          {
            "success": true,
            "expectation_config": {
              "expectation_type": "expect_column_values_to_not_be_null",
              "kwargs": {
                "column": "energy",
                "batch_id": "features-features_dataframe"
              },
              "meta": {}
            },
            "result": {
              "element_count": 179940,
              "unexpected_count": 0,
              "unexpected_percent": 0.0,
              "partial_unexpected_list": [],
              "partial_unexpected_counts": [],
              "partial_unexpected_index_list": []
            },
            "meta": {},
            "exception_info": {
           